In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append("..")

from python.preprocess import *  #only for the categorical variables
from python.OREM import *
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Preprocess

In [23]:
type = "[Conti]"
fn = "yeast-2vs8"

if type == "[Cate]":
    location = "categorical"
if type == "[Conti]":
    location = "continuous"

data = pd.read_csv("../data/"+type+" "+fn+".csv", index_col=0)
data = check_class(data)
# You can see the printed statements have the same results with belows
#data.iloc[:, -1].value_counts()
display(data.head(3))

# of 0: 462
# of 1: 20


,V1,V2,V3,V4,V5,V6,V7,V8,y
1,0.51,0.40,0.56,0.17,0.5,0.5,0.49,0.22,0
2,0.40,0.39,0.60,0.15,0.5,0.0,0.58,0.30,0
3,0.40,0.42,0.57,0.35,0.5,0.0,0.53,0.25,0


In [24]:
# Try this only for the categorical datasets
# 해당 과정은, 범주형 변수 n개를 0부터 n-1까지의 값들로 바꿔준다.
# 따라서, 해당 과정을 거쳐도 컬럼 갯수는 늘어나지 않는다.
df = check_uniqueness(data.iloc[:, :-1])
df['Label'] = data.iloc[:, -1]
df.head(3)

There is at least one column which has different length of unique values.


,V1,V2,V3,V4,V5,V6,V7,V8,Label
1,0.51,0.40,0.56,0.17,0.5,0.5,0.49,0.22,0
2,0.40,0.39,0.60,0.15,0.5,0.0,0.58,0.30,0
3,0.40,0.42,0.57,0.35,0.5,0.0,0.53,0.25,0


# OREM

Before running the codes, please check if the code for the saving works or not.

In [25]:
minclass = 1
df_maj = df[df.iloc[:, -1].values != 1]
df_min = df[df.iloc[:, -1].values == 1]
ratio = 0.5

failed_trial = []
#for trial in tqdm(range(1, 101)):
for trial in tqdm([2, 5, 23, 24, 28, 44, 66, 72]):
    maj_train, maj_test = train_test_split(df_maj, test_size=ratio)
    min_train, min_test = train_test_split(df_min, test_size=ratio)

    train = pd.concat([maj_train, min_train])
    test = pd.concat([maj_test, min_test])

    label = train.iloc[:, -1]
    vc = label.value_counts()
    n_maj = vc[0]
    n_min = vc[1]

    train = train.iloc[:, :-1].copy()
    try:
        newDf, newLabel = OREM(train, list(label), n_maj - n_min)
        newDf['Class'] = newLabel
        #newDf.Class.value_counts()     #check if you want to see if a balanced dataset is generated
        newDf.to_csv("../data/"+location+"/"+fn+"_train"+str(trial)+".csv", index=False)
        test.to_csv("../data/"+location+"/"+fn+"_test"+str(trial)+".csv", index=False)
    except:
        failed_trial.append(trial)
        pass    

100%|██████████| 8/8 [00:00<00:00, 21.53it/s]


In [26]:
failed_trial

[]

In [19]:
if failed_trial: 
    failed_trial2 = []
    for trial in tqdm(failed_trial):
        maj_train, maj_test = train_test_split(df_maj, test_size=ratio)
        min_train, min_test = train_test_split(df_min, test_size=ratio)

        train = pd.concat([maj_train, min_train])
        test = pd.concat([maj_test, min_test])

        label = train.iloc[:, -1]
        vc = label.value_counts()
        n_maj = vc[0]
        n_min = vc[1]

        train = train.iloc[:, :-1].copy()
        try:
            newDf, newLabel = OREM(train, list(label), n_maj - n_min)
            newDf['Class'] = newLabel
            #newDf.Class.value_counts()     #check if you want to see if a balanced dataset is generated
            newDf.to_csv("../data/"+location+"/"+fn+"_train"+str(trial)+".csv", index=False)
            test.to_csv("../data/"+location+"/"+fn+"_test"+str(trial)+".csv", index=False)
        except:
            failed_trial2.append(trial)
            pass    

  0%|          | 0/3 [00:00<?, ?it/s]/Users/hwane/Documents/GitHub/RnR/notebooks/../python/OREM.py:96: RuntimeWarning: divide by zero encountered in scalar divide
  N = int(N/len(AS))
100%|██████████| 3/3 [00:00<00:00, 18.36it/s]
